Encoder:

The RNN class represents a basic RNN, with methods for the forward pass (forward) and step-wise recurrence (step).
The encoder processes the input sequence and passes its last hidden state to the decoder.
Decoder:

The decoder is another RNN, initialized with the encoder’s final hidden state as its initial hidden state.
It sequentially takes in each token from the target sequence and generates an output for each time step.
Forward and Backward Pass:

In the forward pass, the encoder processes the input sequence, and the decoder uses the encoder’s final hidden state to produce outputs for each time step in the target sequence.
During backpropagation, the model computes gradients for the decoder and propagates errors backward through time for each hidden state.
Training:

The train method repeatedly calls the forward and backward passes over multiple epochs, updating weights based on gradients at each step.

In [ ]:
import numpy as np

class Seq2Seq:
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.01):
        self.encoder = RNN(input_dim, hidden_dim)
        self.decoder = RNN(output_dim, hidden_dim)
        self.output_layer = np.random.randn(output_dim, hidden_dim) * 0.01
        self.output_bias = np.zeros((output_dim, 1))
        self.hidden_dim = hidden_dim
        self.learning_rate = learning_rate

    def forward(self, input_seq, target_seq):
        # Encoder forward pass
        encoder_hidden_states = self.encoder.forward(input_seq)

        # Decoder forward pass (using last hidden state of encoder)
        decoder_hidden = encoder_hidden_states[-1]
        decoder_outputs = []
        for target in target_seq:
            decoder_hidden = self.decoder.step(target, decoder_hidden)
            output = np.dot(self.output_layer, decoder_hidden) + self.output_bias
            decoder_outputs.append(output)

        return encoder_hidden_states, decoder_outputs

    def backward(self, input_seq, target_seq, encoder_hidden_states, decoder_outputs):
        # Initialize gradients
        dW_out = np.zeros_like(self.output_layer)
        dB_out = np.zeros_like(self.output_bias)

        # Initialize the gradients for decoder hidden state
        dh_decoder = np.zeros((self.hidden_dim, 1))

        # Backprop through decoder time steps
        for t in reversed(range(len(target_seq))):
            dy = decoder_outputs[t] - target_seq[t].reshape(-1, 1)
            dW_out += np.dot(dy, encoder_hidden_states[-1].T)
            dB_out += dy

            dh_decoder += np.dot(self.output_layer.T, dy)
            dh_decoder = self.decoder.backward_step(dh_decoder)

        # Backprop through encoder time steps
        dh_encoder = dh_decoder
        for t in reversed(range(len(input_seq))):
            dh_encoder = self.encoder.backward_step(dh_encoder)

        # Update weights
        self.output_layer -= self.learning_rate * dW_out
        self.output_bias -= self.learning_rate * dB_out

    def train(self, input_seq, target_seq, epochs=100):
        for epoch in range(epochs):
            encoder_hidden_states, decoder_outputs = self.forward(input_seq, target_seq)
            self.backward(input_seq, target_seq, encoder_hidden_states, decoder_outputs)
            if epoch % 10 == 0:
                loss = sum((target.reshape(-1, 1) - output) ** 2 for target, output in zip(target_seq, decoder_outputs))
                print(f'Epoch {epoch}, Loss: {np.sum(loss)}')


class RNN:
    def __init__(self, input_dim, hidden_dim):
        self.hidden_dim = hidden_dim
        self.Wx = np.random.randn(hidden_dim, input_dim) * 0.01
        self.Wh = np.random.randn(hidden_dim, hidden_dim) * 0.01
        self.bh = np.zeros((hidden_dim, 1))

    def forward(self, inputs):
        self.hidden_states = []
        h_prev = np.zeros((self.hidden_dim, 1))
        for x in inputs:
            h_prev = self.step(x, h_prev)
            self.hidden_states.append(h_prev)
        return self.hidden_states

    def step(self, x, h_prev):
        h_next = np.tanh(np.dot(self.Wx, x.reshape(-1, 1)) + np.dot(self.Wh, h_prev) + self.bh)
        return h_next

    def backward_step(self, dh_next):
        # Backpropagation for a single RNN step (time-step in sequence)
        # This should update weights but here it returns `dh` for backprop to previous time steps
        return np.dot(self.Wh.T, dh_next)

# Example input and target sequences
input_seq = [np.array([0.5, -0.2]), np.array([0.1, 0.8]), np.array([-0.3, 0.2])]
target_seq = [np.array([0.1]), np.array([0.4]), np.array([-0.1])]

# Initialize Seq2Seq model and train
seq2seq = Seq2Seq(input_dim=2, hidden_dim=4, output_dim=1, learning_rate=0.01)
seq2seq.train(input_seq, target_seq, epochs=100)
